In [1]:
import sys
import numpy as np
# import hipsternet.neuralnet as nn
import hipsternet.RNN as rnn
import hipsternet.solver as solver

In [2]:
time_step = 10
n_iter = 100000 # epochs
alpha = 1e-3
print_after = 1000
H = 64

In [3]:
# if __name__ == '__main__':
with open('data/text_data/japan.txt', 'r') as f:
    txt = f.read()

    X = []
    y = []

    char_to_idx = {char: i for i, char in enumerate(set(txt))}
    idx_to_char = {i: char for i, char in enumerate(set(txt))}

    X = np.array([char_to_idx[x] for x in txt])
    y = [char_to_idx[x] for x in txt[1:]]
    y.append(char_to_idx['.'])
    y = np.array(y)

In [4]:
vocab_size = len(char_to_idx)

In [5]:
# net = nn.LSTM(vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)
# net = nn.RNN(vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)
net = rnn.RNN(vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)
# net = nn.GRU(vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

In [6]:
solver.adam_rnn(
    net, X, y,
    alpha=alpha,
    mb_size=time_step,
    n_iter=n_iter,
    print_after=print_after
)

Iter-1000 loss: 3.4707
 'j7ddw facod bn Ng eys tae phd a Jca G des in En e lhd , tin 1 r hs tliphcasiE 1o inc aneng ieev ta


Iter-2000 loss: 2.8378
ouseag mulponhly canwienAland anlesony pixgl rer alestictmes nrth andarJathe shg endevl,rris eamanro


Iter-3000 loss: 2.4064
re  akeos in rinldgloci GlonSee ias civeccint in tithe Inrthi n, hocbto tmoes on, Euend.and is. Seur


Iter-4000 loss: 2.1200
u, colingtstl Japan's corkasil. UenirtU-stron P Eapere JfpooledP c2upsrl ropoded loran, and 5xpmeune


Iter-5000 loss: 1.9176
co thed E par haraede in turcamo f cmaned an i9gl inith d9dar-yesgurilako-uanu Morlasingppoicrgo the


Iter-6000 loss: 1.7784
oen iyts thecRus thitMoc ty pekdor orse aue Gh ist. io the rorlalIofwrl worctuit a eseans cethe wid 


Iter-7000 loss: 1.6568
an ockalupiry io topen in Japan-(aresou hiket Siak. The an moppoxof wer the Eurc-Tal herc. Japan enc


Iter-8000 loss: 1.5653
in th in0thKy sr. Japan inkoinn-bbento-JaSen'f ingeann-t, an-kokeW e the Seases , at the Sovrig

Iter-25000 loss: 0.9582
ra ed a perchasekio Ahighisnd Indered paund the wern-khounthe and iestorectalapinted an expanivite i


Iter-26000 loss: 0.9317
 great pulios wiftrry Winj-le'n at os Wistr Sealecslly. Anfinn-higech sts otenturg, the the tore ous


Iter-27000 loss: 0.9164
hesinaby oulect orgest epled. The werlo, Khica is andes und itore war Nively-935 mangurlacy oplrtate


Iter-28000 loss: 0.8867
ved The an ing ho the Emperea us NeDPy pint on ean . Japan, which an the so thi5S, mo ar ontto heghi


Iter-29000 loss: 0.8798
ighest-ret an or tha cadeste morureanaid peepeigsry is a medper, ul c9.8-(D0ban Ky Broncevino-re tit


Iter-30000 loss: 0.8600
f of itonekyinacgiea irallowtel chuleciner itha ised an imel lars axd rise G7s and rofpth tor as , w


Iter-31000 loss: 0.8431
alria orof try cermiceceinision ingesthires arcarealy isendutin. Japanace ofly. Toky pawer. Thita is


Iter-32000 loss: 0.8200
kokr uvivine Sea, it the Sea of in byuperchisiry Bstur d peracener Statopen an a loCE. 

Iter-49000 loss: 0.6537
 in E8by pured frimtor to-ropen arcy bund JapanecIno, whisunded wored As Westilaclues a ghes mains a


Iter-50000 loss: 0.6384
s cins ingest wricinal of tho and iansidendicitirn mititalivenilial constion ar an is the Warcy 12CW


Iter-51000 loss: 0.6193
e Emperor as s rct 201d at ar on tho the sideres, the conhich ind eaplould oresthis tho a the Meioli


Iter-52000 loss: 0.6244
ty. Arch inliat lovexts feustrinsirg son, whica es iflande945 Kored hiping the hore four higheg part


Iter-53000 loss: 0.6160
an has puresodeppentmonthi anan "Sthe Gle Pnymen in tet laegshi arloffratmoins ins tho audsra, parch


Iter-54000 loss: 0.6199
my in 1868 GDoes urobul and the wowl -lakasy Chila, nhisy wymitiness on in ihitat Tokyonan enokiry w


Iter-55000 loss: 0.6184
wo cedwith. The kare, and the world's fourty-lmounhitholat's latintr Tokat of Wirthita in 1941, whic


Iter-56000 loss: 0.6091
e world's pentist ortitalima. Japan has can'mpeagse fiul compelisod legest arc faist in

Iter-73000 loss: 0.4925
arnan foresud to iem inst of the nan ankanji the firso Japan ason ked peateo Japentersin6, wandith t


Iter-74000 loss: 0.5115
s. Japan War Japan os elcanesy fftic G18, Intros onlaided liog at of islaban Japan, wh cousorlded Ja


Iter-75000 loss: 0.4943
to edic es in turodextun Intereasing gibolloper al Csinted in the tored in the ner. The countromed i


Iter-76000 loss: 0.5128
 Nahase . The Emperosintulint iolled iopod inted Astr in the G8, China Seasided into part GP eand a 


Iter-77000 loss: 0.4928
ed bal China, followed byupured Japan the world's coures tiven crest ind and Humollectur to tee dect


Iter-78000 loss: 0.4882
alatinin main, inatek. up rtellainted Simc owarcN ported ivelep. umbintored a ghes hish has ofrch ma


Iter-79000 loss: 0.4837
onHuland aadicinar cat oritoliWen ippeesich worlind opuren, wiont d wat on ofy sureroliot of the eal


Iter-80000 loss: 0.4787
hean-sevikele mp rest. Nipand oputing ad Areaka, peasien. Japan eas the world's largest

Iter-97000 loss: 0.4116
ke wo the Emperor as a sod fathorovittor city in the witho Japan hokkopyI sod War II le live ivininy


Iter-98000 loss: 0.4144
he world, and rarco wity and enonist fer.-Japanthits untropentery cfocmaind inflyme tory inted lire 


Iter-99000 loss: 0.4184
 "U35upand warly 9uroma, waslowat to intarcentar collldion of 1261 f rorla. Amallowod Wiss reages To


Iter-100000 loss: 0.4000
the G7, the G8, and the Globol lais and Ifeleates in the OECD and the world's largest mitelunedg ppe




In [ ]:
=========================================================================
Iter-100000 loss: 0.3913
=========================================================================
the 1pman a inusy rotory surrowho tOkike upporter wor peven des The easty porterized itivecend perce
=========================================================================


Out[7]:
<hipsternet.neuralnet.RNN at 0x7fc2f5182e80>

In [ ]:
=========================================================================
Iter-100000 loss: 0.1615
=========================================================================
the name of the 12th century until 1868, Japan was ruled by successime Wer of 1941, which country is
=========================================================================


Out[16]:
<hipsternet.neuralnet.LSTM at 0x7f67fe0c9978>

In [ ]:
=========================================================================
Iter-100000 loss: 0.1333
=========================================================================
the Coun resilate a in the population. Russided in the OECD and the world's fourth-largest econompic
=========================================================================


Out[10]:
<hipsternet.neuralnet.GRU at 0x7fc2cec1e1d0>